In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
# 1) Remove any stale install, then install your local package (editable)
%pip uninstall -y surveytools
%pip install -e "/Workspace/teams/survey_team/Core_automation/Correct code/surveytools" --verbose


In [0]:
# 2) Restart the Python worker in THIS notebook
dbutils.library.restartPython()


In [0]:
# 3) Confirm Python can see the installed package
import sys, importlib.metadata as im

print("Top sys.path[0..3]:")
print("\n".join(sys.path[:4]))

# Is the distribution installed into this environment?
names = [d.metadata.get("Name","").lower() for d in im.distributions()]
print("\n'surveytools' present in installed dists? ->", "surveytools" in names)

# Try importing
from surveytools.types import SummaryConfig
SummaryConfig()


In [0]:
import sys, os

# 4) Add your package's src folder to Python's search path for THIS notebook session
sys.path.insert(0, "/Workspace/teams/survey_team/Core_automation/Correct code/surveytools/src")

# 5) Now import
from surveytools.types import SummaryConfig
SummaryConfig()


In [0]:
# import bootstrap (keep at top of notebooks that use surveytools)
import sys
SRC = "/Workspace/teams/survey_team/Core_automation/Correct code/surveytools/src"
if SRC not in sys.path:
    sys.path.insert(0, SRC)


In [0]:
from surveytools.constants import ZC_TURQUOISE, ZERO_ALIASES, PRESETS
print("One color:", ZC_TURQUOISE)
print("ZERO_ALIASES count:", len(ZERO_ALIASES))
print("Preset keys:", list(PRESETS.keys()))


In [0]:
import pandas as pd
from surveytools.cleaning import fix_df_text, fill_client_tokens, is_likert_candidate, norm_key, normalized

# tiny sample
df = pd.DataFrame({
    "client_name": ["Zebra City", None],
    "Q1: How is %client_name%? (general)": ["Very good", "very  good  "],
    "comment": ["I love %client_name%!", None]
})

fixed = fix_df_text(df)
filled = fill_client_tokens(fixed)

print("Columns after fill_client_tokens:")
print(list(filled.columns))
print("\nCell example:", filled.loc[0, "comment"])
print("\nLikert candidate?", is_likert_candidate(filled["Q1: How is Zebra City? (general)"]))
print("norm_key('  Very   Good '):", norm_key("  Very   Good "))
print("normalized('  keep Case  '):", normalized("  keep Case  "))


In [0]:
with open("src/surveytools/summarize.py", "r") as f:
    for i, line in enumerate(f, start=1):
        if "\t" in line:
            print(f"Line {i} contains a tab character.")

In [0]:
import importlib, surveytools.summarize
importlib.reload(surveytools.summarize)
from surveytools.summarize import TableSummarizer


In [0]:
import pandas as pd
from surveytools.summarize import TableSummarizer
from surveytools.types import SummaryConfig

# toy dataset
df = pd.DataFrame({
    "response_id": [1,2,3,4,5,6],
    "latest_weight": [1,1,1,1,1,1],
    "gender (demographics)": ["Female","Male","Female","Female","Male","Female"],
    # a scale-like column (values already in "label: number" format)
    "Q1 Satisfaction (general)": ["Very satisfied: 5","Somewhat satisfied: 4","Neutral: 3",
                                  "Somewhat satisfied: 4","Very satisfied: 5","Neutral: 3"],
    # a single-choice column
    "Q2 Favorite Fruit": ["Apple","Banana","Apple","Orange","Apple","Banana"]
})

summ = TableSummarizer(df, SummaryConfig())
print("Group options:", summ.group_options)
print("Question options:", summ.question_options)

# Scale example
res1 = summ.summarize_overall_and_demo("Q1 Satisfaction (general)", ["gender (demographics)"])
print("\n--- Q1 summary (head) ---")
print(res1.head())

# Choice example
res2 = summ.summarize_overall_and_demo("Q2 Favorite Fruit", ["gender (demographics)"])
print("\n--- Q2 summary (head) ---")
print(res2.head())


In [0]:
%pip install "ipywidgets>=8,<9" "ipython>=8"
# then click "Restart Python" when prompted



In [0]:
from importlib import reload
import surveytools.likert_mapper as lm_mod
reload(lm_mod)  # pick up your latest edit

import pandas as pd
from surveytools.likert_mapper import LikertMapper

df = pd.DataFrame({
    "Q1 Satisfaction (general)": ["very satisfied","somewhat satisfied","neutral","prefer not to say", None]
})

lm = LikertMapper(df)
print("LikertMapper imported. Widgets available?", lm_mod.W is not None)

# (Optional) simulate a quick mapping without UI:
lm.SHARED_MAP = {
    "very satisfied": 5,
    "somewhat satisfied": 4,
    "neutral": 3,
}
# Apply mapping to that one column (headless)
lm._apply_mapping_to_column("Q1 Satisfaction (general)")
df.head()


In [0]:
import pandas as pd
from surveytools.bulk_assistant import bulk_export_headless
from surveytools.types import SummaryConfig

# Example df (replace with your real one)
df = pd.DataFrame({
    "response_id":[1,2,3,4,5,6],
    "gender (demographics)":["Female","Male","Female","Male","Female","Male"],
    "Q1 Satisfaction (general)":[
        "very satisfied: 5","somewhat satisfied: 4","neutral: 3",
        "very satisfied: 5","somewhat satisfied: 4","prefer not to say: 0"
    ],
    "Q2 Favorite Fruit":[
        "Apple","Apple|Banana","Banana","Orange","Apple","Other: Grapes"
    ],
    "latest_weight":[1,1,1,1,1,1],
})

out = bulk_export_headless(
    df,
    groups=["gender (demographics)"],
    filename="All_Questions_Summary.xlsx",
    config=SummaryConfig(output_mode="percent", decimals=1),
)
print("Wrote:", out)


In [0]:
import pandas as pd
from surveytools.report_assistant import export_single_question, save_overall_chart

# df = ... load your real dataframe ...

# Save per-question summary to an Excel sheet:
path, table = export_single_question(
    df,
    question="Q1 Satisfaction (general)",
    groups=["gender (demographics)"],
    filename="Summary.xlsx"
)
print("Wrote:", path)
display(table.head())  # optional

# Save a simple overall chart for that question:
png = save_overall_chart(df, "Q1 Satisfaction (general)", path="overall_q1.png")
print("Chart saved:", png)


In [0]:
%cd /Workspace/teams/survey_team/Core_automation/Correct\ code/surveytools
%pip install -e ".[widgets]"


In [0]:
dbutils.library.restartPython()

In [0]:
%sh surveytools --help


In [0]:
%sh python -m surveytools.cli --help


In [0]:
%sh ls -l "/Workspace/teams/survey_team/Core_automation/Correct code/surveytools/src/surveytools"
